In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,GRU, Dropout, Dense, Conv1D, MaxPooling1D, Flatten
from tensorflow.keras.optimizers import Adam
from imblearn.over_sampling import RandomOverSampler


2024-09-11 10:29:16.840752: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-09-11 10:29:16.901831: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-11 10:29:16.930230: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-11 10:29:16.938690: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-11 10:29:16.980253: I tensorflow/core/platform/cpu_feature_guar

# Teste sem torque

In [2]:
df = pd.read_csv("data/Merge_Falhas_Resultados.csv")
df["S_GROUP_ID_1"]

0        1
1        0
2        4
3        2
4        1
        ..
40149    4
40150    0
40151    4
40152    2
40153    2
Name: S_GROUP_ID_1, Length: 40154, dtype: int64

In [3]:
# Converter a coluna para binário
df['S_GROUP_ID_1'] = (df['S_GROUP_ID_1'] > 0).astype(int)
df['S_GROUP_ID_1']

0        1
1        0
2        1
3        1
4        1
        ..
40149    1
40150    0
40151    1
40152    1
40153    1
Name: S_GROUP_ID_1, Length: 40154, dtype: int64

In [4]:
# Separando as features (X) e o target (y)
X = df.drop(columns=['S_GROUP_ID_1', 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
y = df['S_GROUP_ID_1']

In [5]:
# Separando em dados de treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [6]:
# Converte X_train e X_test para arrays NumPy, caso ainda não sejam.
X_train = np.array(X_train)
X_test = np.array(X_test)

# Reestrutura X_train e X_test para ter 3 dimensões.
# A nova forma do array será (n_samples, n_features, 1)
X_train = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

In [7]:
# Construção do modelo com LSTM
model_1 = Sequential()

model_1.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_1.add(LSTM(50, activation='relu'))
model_1.add(Dense(1))

model_1.compile(optimizer='adam', loss='mse')

I0000 00:00:1725996846.963590  512330 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-10 16:34:06.987229: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/barretob/Modelos/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
# Treinamento do modelo
model_1.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100


1004/1004 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 1304.3783 - val_loss: 2.3630
Epoch 2/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 1.5141 - val_loss: 0.7995
Epoch 3/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 6s 6ms/step - loss: 0.5506 - val_loss: 0.2872
Epoch 4/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 7s 6ms/step - loss: 29158.4648 - val_loss: 84.3931
Epoch 5/100
  11/1004 ━━━━━━━━━━━━━━━━━━━━ 5s 5ms/step - loss: 236.5452

KeyboardInterrupt: 

In [ ]:
#Prever os dados de teste
y_pred_1 = model_1.predict(X_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_1 = (y_pred_1 > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y_test, y_pred_classes_1)
precision = precision_score(y_test, y_pred_classes_1)
recall = recall_score(y_test, y_pred_classes_1)
f1 = f1_score(y_test, y_pred_classes_1)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

# # Matriz de Confusão
# cm = confusion_matrix(y_test, y_pred_classes)

# # Exibindo a Matriz de Confusão
# disp = ConfusionMatrixDisplay(confusion_matrix=cm)
# disp.plot(cmap=plt.cm.Blues)
# plt.title('Confusion Matrix')
# plt.show()

251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.6210
Precision: 0.6220
Recall: 0.9972
F1-Score: 0.7661


In [ ]:
df["S_GROUP_ID_1"].describe()

count    40154.000000
mean         0.621507
std          0.485017
min          0.000000
25%          0.000000
50%          1.000000
75%          1.000000
max          1.000000
Name: S_GROUP_ID_1, dtype: float64

In [ ]:
# Construção do modelo com GRU
model_2 = Sequential()

model_2.add(GRU(50, activation='relu', return_sequences=True, input_shape=(X_train.shape[1], 1)))
model_2.add(GRU(50, activation='relu'))
model_2.add(Dense(1, activation='sigmoid'))

model_2.compile(optimizer='adam', loss='binary_crossentropy')

/home/barretob/Modelos/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [ ]:
# Treinamento do modelo
model_2.fit(X_train, y_train, epochs=100, batch_size=32, validation_data=(X_test, y_test))

Epoch 1/100


1004/1004 ━━━━━━━━━━━━━━━━━━━━ 8s 7ms/step - loss: 0.7857 - val_loss: 0.6907
Epoch 2/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.6835 - val_loss: 0.6666
Epoch 3/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.6734 - val_loss: 0.6625
Epoch 4/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.6679 - val_loss: 0.6700
Epoch 5/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.6660 - val_loss: 0.6634
Epoch 6/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.6642 - val_loss: 0.6625
Epoch 7/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.6630 - val_loss: 0.6619
Epoch 8/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.6634 - val_loss: 0.6618
Epoch 9/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.6736 - val_loss: 0.6628
Epoch 10/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.6638 - val_loss: 0.6589
Epoch 11/100
1004/1004 ━━━━━━━━━━━━━━━━━━━━ 7s 7ms/step - loss: 0.6580 - val_loss: 0.6576
Epoch 12/100
1004/1004 ━━━━━━━━

In [ ]:
# Prever os dados de teste
y_pred_2 = model_2.predict(X_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_2 = (y_pred_2 > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y_test, y_pred_classes_2)
precision = precision_score(y_test, y_pred_classes_2)
recall = recall_score(y_test, y_pred_classes_2)
f1 = f1_score(y_test, y_pred_classes_2)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

251/251 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.6218
Precision: 0.6297
Recall: 0.9526
F1-Score: 0.7582


# Teste com torque

In [3]:
df2 = pd.read_csv("data/df_torques_falhas.csv")
df2.columns

Index(['KNR', '%', 'CLICKS', 'DEG', 'GRAD', 'NM', 'V', '\U00B0', 'KG', 'MIN',
       'MM', 'ID1NAME', 'ID1SOK', 'ID1SNOK', 'ID1DATA', 'ID2NAME', 'ID2SOK',
       'ID2SNOK', 'ID2DATA', 'ID718NAME', 'ID718SOK', 'ID718SNOK', 'ID718DATA',
       'S_GROUP_ID_-2', 'S_GROUP_ID_1', 'S_GROUP_ID_2', 'S_GROUP_ID_4',
       'S_GROUP_ID_5', 'S_GROUP_ID_133', 'S_GROUP_ID_137', 'S_GROUP_ID_140',
       'S_GROUP_ID_9830946', 'COR_0Q0Q', 'COR_0QA1', 'COR_2R2R', 'COR_2RA1',
       'COR_5T5T', 'COR_5TA1', 'COR_6K6K', 'COR_6KA1', 'COR_6UA1', 'COR_A1A1',
       'COR_K2A1', 'COR_K2K2', 'COR_O7O7', 'MOTOR_CWL', 'MOTOR_CWS',
       'MOTOR_DHS', 'MOTOR_DRP'],
      dtype='object')

In [4]:
# Converter a coluna para binário
df2['S_GROUP_ID_1'] = (df2['S_GROUP_ID_1'] > 0).astype(int)
df2['S_GROUP_ID_1']



0        0
1        0
2        0
3        0
4        0
        ..
49186    1
49187    0
49188    1
49189    1
49190    1
Name: S_GROUP_ID_1, Length: 49191, dtype: int64

In [5]:
df2 = df2.dropna()
df2.shape

(40152, 49)

In [6]:
# checando quantidade de valores 0
df2_filtrado = df2[df2['S_GROUP_ID_1'] == 0]
df2_filtrado.shape
# df2.shape
# df2['S_GROUP_ID_1'].unique()

(15207, 49)

In [7]:
# checando quantidade de valores 1
df2_filtrado = df2[df2['S_GROUP_ID_1'] == 1]
df2_filtrado.shape

(24945, 49)

In [8]:
df2['S_GROUP_ID_1'].unique()

array([1, 0])

In [9]:
# Separando as features (X) e o target (y)
X2 = df2.drop(columns=['S_GROUP_ID_1', 'KNR'])  # 'KNR' é apenas um identificador, então deve ser removido
y2 = df2['S_GROUP_ID_1']
print(np.isnan(X2).sum())


%                     0
CLICKS                0
DEG                   0
GRAD                  0
NM                    0
V                     0
\U00B0                0
KG                    0
MIN                   0
MM                    0
ID1NAME               0
ID1SOK                0
ID1SNOK               0
ID1DATA               0
ID2NAME               0
ID2SOK                0
ID2SNOK               0
ID2DATA               0
ID718NAME             0
ID718SOK              0
ID718SNOK             0
ID718DATA             0
S_GROUP_ID_-2         0
S_GROUP_ID_2          0
S_GROUP_ID_4          0
S_GROUP_ID_5          0
S_GROUP_ID_133        0
S_GROUP_ID_137        0
S_GROUP_ID_140        0
S_GROUP_ID_9830946    0
COR_0Q0Q              0
COR_0QA1              0
COR_2R2R              0
COR_2RA1              0
COR_5T5T              0
COR_5TA1              0
COR_6K6K              0
COR_6KA1              0
COR_6UA1              0
COR_A1A1              0
COR_K2A1              0
COR_K2K2        

In [10]:
from imblearn.over_sampling import SMOTE


# Aplicar SMOTE
smote = SMOTE(random_state=42)
X2, y2 = smote.fit_resample(X2, y2)

# O novo dataset estará balanceado com amostras sintéticas da classe minoritária



In [11]:
# Separando em dados de treino e teste
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.2, random_state=42)
print(np.isnan(X2_train).sum())

%                     0
CLICKS                0
DEG                   0
GRAD                  0
NM                    0
V                     0
\U00B0                0
KG                    0
MIN                   0
MM                    0
ID1NAME               0
ID1SOK                0
ID1SNOK               0
ID1DATA               0
ID2NAME               0
ID2SOK                0
ID2SNOK               0
ID2DATA               0
ID718NAME             0
ID718SOK              0
ID718SNOK             0
ID718DATA             0
S_GROUP_ID_-2         0
S_GROUP_ID_2          0
S_GROUP_ID_4          0
S_GROUP_ID_5          0
S_GROUP_ID_133        0
S_GROUP_ID_137        0
S_GROUP_ID_140        0
S_GROUP_ID_9830946    0
COR_0Q0Q              0
COR_0QA1              0
COR_2R2R              0
COR_2RA1              0
COR_5T5T              0
COR_5TA1              0
COR_6K6K              0
COR_6KA1              0
COR_6UA1              0
COR_A1A1              0
COR_K2A1              0
COR_K2K2        

In [12]:
# Converte X_train e X_test para arrays NumPy, caso ainda não sejam.
X2_train = np.array(X2_train)
X2_test = np.array(X2_test)
# Reestrutura X_train e X_test para ter 3 dimensões.
# A nova forma do array será (n_samples, n_features, 1)
X2_train = X2_train.reshape((X2_train.shape[0], X2_train.shape[1], 1))
X2_test = X2_test.reshape((X2_test.shape[0], X2_test.shape[1], 1))


In [13]:
print(np.isnan(X2_train).sum())
print(np.isinf(X2_train).sum())


0
0


In [14]:
# Construção do modelo com LSTM
model_1_2 = Sequential()

model_1_2.add(LSTM(50, activation='relu', return_sequences=True, input_shape=(X2_train.shape[1], 1)))
model_1_2.add(LSTM(50, activation='relu'))
model_1_2.add(Dense(1))

model_1_2.compile(optimizer='adam', loss='mse')

I0000 00:00:1726061366.164796  549561 cuda_executor.cc:1015] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-11 10:29:26.242080: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2343] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
/home/barretob/Modelos/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
# Treinamento do modelo
model_1_2.fit(X2_train, y2_train, epochs=100, batch_size=32, validation_data=(X2_test, y2_test))

Epoch 1/100


1248/1248 ━━━━━━━━━━━━━━━━━━━━ 11s 8ms/step - loss: 943.2964 - val_loss: 3.7710
Epoch 2/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 8.8520 - val_loss: 1.4721
Epoch 3/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 16.1435 - val_loss: 0.8976
Epoch 4/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 1.7224 - val_loss: 0.4002
Epoch 5/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.8073 - val_loss: 0.2648
Epoch 6/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.2692 - val_loss: 0.3435
Epoch 7/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.3177 - val_loss: 0.2759
Epoch 8/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.7625 - val_loss: 0.3095
Epoch 9/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.2903 - val_loss: 0.2521
Epoch 10/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.2777 - val_loss: 0.2753
Epoch 11/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 10s 8ms/step - loss: 0.2723 - val_loss: 0.2552
Epoch 12/100
1248

In [16]:
#Prever os dados de teste
y_pred_1_2 = model_1_2.predict(X2_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_1_2 = (y_pred_1_2 > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y2_test, y_pred_classes_1_2)
precision = precision_score(y2_test, y_pred_classes_1_2)
recall = recall_score(y2_test, y_pred_classes_1_2)
f1 = f1_score(y2_test, y_pred_classes_1_2)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.6615
Precision: 0.5976
Recall: 0.9860
F1-Score: 0.7442


In [17]:
# Construção do modelo com GRU
model_2_2 = Sequential()

model_2_2.add(GRU(50, activation='relu', return_sequences=True, input_shape=(X2_train.shape[1], 1)))
model_2_2.add(GRU(50, activation='relu'))
model_2_2.add(Dense(1, activation='sigmoid'))

model_2_2.compile(optimizer='adam', loss='binary_crossentropy')

/home/barretob/Modelos/.venv/lib/python3.10/site-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [18]:
# Treinamento do modelo
model_2_2.fit(X2_train, y2_train, epochs=100, batch_size=32, validation_data=(X2_test, y2_test))

Epoch 1/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 13s 10ms/step - loss: 0.8845 - val_loss: 0.6990
Epoch 2/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.7106 - val_loss: 0.7443
Epoch 3/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.7157 - val_loss: 0.6938
Epoch 4/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.7036 - val_loss: 0.7024
Epoch 5/100
1170/1248 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.7043

1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.7041 - val_loss: 0.7167
Epoch 6/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.6990 - val_loss: 0.7443
Epoch 7/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.6987 - val_loss: 0.7075
Epoch 8/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.6979 - val_loss: 0.6945
Epoch 9/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.6951 - val_loss: 0.6875
Epoch 10/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.6888 - val_loss: 0.6966
Epoch 11/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.6926 - val_loss: 0.6889
Epoch 12/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.6654 - val_loss: 0.6287
Epoch 13/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.6293 - val_loss: 0.6241
Epoch 14/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.6176 - val_loss: 0.6111
Epoch 15/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - loss: 0.6090 - val_loss: 0.6043
Epoch

In [19]:
# Prever os dados de teste
y_pred_2_2 = model_2_2.predict(X2_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_2_2 = (y_pred_2_2 > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y2_test, y_pred_classes_2_2)
precision = precision_score(y2_test, y_pred_classes_2_2)
recall = recall_score(y2_test, y_pred_classes_2_2)
f1 = f1_score(y2_test, y_pred_classes_2_2)

print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

312/312 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step
Accuracy: 0.6730
Precision: 0.6171
Recall: 0.9097
F1-Score: 0.7353


In [20]:
# Definir o modelo CNN
model_3 = Sequential()

# Primeira camada convolucional
model_3.add(Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(X2_train.shape[1], 1)))
model_3.add(MaxPooling1D(pool_size=2))
model_3.add(Dropout(0.3))

# Segunda camada convolucional
model_3.add(Conv1D(filters=64, kernel_size=2, activation='relu'))
model_3.add(MaxPooling1D(pool_size=2))
model_3.add(Dropout(0.3))

# Flatten para converter dados 2D em 1D
model_3.add(Flatten())

# Camada totalmente conectada
model_3.add(Dense(64, activation='relu'))
model_3.add(Dropout(0.5))

# Camada de saída para classificação binária
model_3.add(Dense(1, activation='sigmoid'))

# Compilação do modelo
model_3.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])


/home/barretob/Modelos/.venv/lib/python3.10/site-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [21]:
# Treinamento do modelo
model_3.fit(X2_train, y2_train, epochs=100, batch_size=32, validation_data=(X2_test, y2_test))

Epoch 1/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.5013 - loss: 3.2792 - val_accuracy: 0.4984 - val_loss: 0.6932
Epoch 2/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4979 - loss: 0.6967 - val_accuracy: 0.4994 - val_loss: 0.6931
Epoch 3/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5008 - loss: 0.6933 - val_accuracy: 0.5019 - val_loss: 0.6930
Epoch 4/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4983 - loss: 0.6943 - val_accuracy: 0.5006 - val_loss: 0.6932
Epoch 5/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5020 - loss: 0.6933 - val_accuracy: 0.4994 - val_loss: 0.6930
Epoch 6/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4976 - loss: 0.6932 - val_accuracy: 0.4994 - val_loss: 0.6931
Epoch 7/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4999 - loss: 0.6932 - val_accuracy: 0.4994 - val_loss: 0.6930
Epoch 8/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.5050 - loss: 0

1248/1248 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.4930 - loss: 0.6928 - val_accuracy: 0.4994 - val_loss: 0.6929
Epoch 78/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.4993 - loss: 0.6928 - val_accuracy: 0.5009 - val_loss: 0.6930
Epoch 79/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.4984 - loss: 0.6932 - val_accuracy: 0.4994 - val_loss: 0.6930
Epoch 80/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4981 - loss: 0.6929 - val_accuracy: 0.5009 - val_loss: 0.6930
Epoch 81/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4939 - loss: 0.6926 - val_accuracy: 0.5007 - val_loss: 0.6930
Epoch 82/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 2s 1ms/step - accuracy: 0.4998 - loss: 0.6929 - val_accuracy: 0.4994 - val_loss: 0.6930
Epoch 83/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4989 - loss: 0.6928 - val_accuracy: 0.5009 - val_loss: 0.6930
Epoch 84/100
1248/1248 ━━━━━━━━━━━━━━━━━━━━ 1s 1ms/step - accuracy: 0.4987 - loss: 0.6928

In [22]:
# Prever os dados de teste
y_pred_3 = model_3.predict(X2_test)

# Converter as probabilidades em classes binárias (0 ou 1)
y_pred_classes_3 = (y_pred_3 > 0.5).astype(int)

# Calcular as principais métricas
accuracy = accuracy_score(y2_test, y_pred_classes_3)
precision = precision_score(y2_test, y_pred_classes_3)
recall = recall_score(y2_test, y_pred_classes_3)
f1 = f1_score(y2_test, y_pred_classes_3)

# Exibir as métricas
print(f"Accuracy: {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1-Score: {f1:.4f}")

312/312 ━━━━━━━━━━━━━━━━━━━━ 0s 523us/step
Accuracy: 0.5012
Precision: 0.6250
Recall: 0.0030
F1-Score: 0.0060
